# Customer Churn Prediction in Telecommunication

## 1. Import required libraries

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, precision_score, recall_score
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from bayes_opt import BayesianOptimization
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 2. Load and preprocess the data

#### Load the dataset into a Pandas dataframe

In [2]:
train = pd.read_csv('data/cell2celltrain.csv')
test = pd.read_csv('data/cell2cellholdout.csv')

df = pd.concat([train, test], ignore_index=True)
df = shuffle(df)

#### Understanding the dataset

In [3]:
# Display the first 5 rows of the dataframe
df.head()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
48970,3384658,No,51.29,955.0,45.0,0.00,31.0,0.0,242.0,-12.3,...,0,4,No,0,Unknown,No,3-Good,Other,Other,Unknown
8816,3069482,No,83.54,338.0,67.0,0.25,58.0,0.0,42.0,25.8,...,0,5,No,0,Unknown,No,3-Good,Suburban,Other,Yes
28036,3222010,No,71.70,928.0,60.0,1.24,39.0,3.1,-130.0,-12.1,...,0,8,No,0,150,No,2-High,Suburban,Professional,No
46308,3366282,No,34.99,158.0,45.0,0.00,0.0,0.0,-36.0,0.0,...,0,0,No,0,Unknown,No,2-High,Town,Other,Unknown
40875,3325714,No,47.27,807.0,52.0,0.00,1.0,0.0,-807.0,-141.2,...,0,1,No,0,30,No,5-Low,Town,Other,No


In [4]:
# Display summary statistics for the numerical columns
df.describe()

,CustomerID,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,...,Handsets,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,RetentionCalls,RetentionOffersAccepted,ReferralsMadeBySubscriber,IncomeGroup,AdjustmentsToCreditRating
count,7.104700e+04,70831.000000,70831.000000,70831.000000,70831.000000,70831.000000,70831.000000,70545.000000,70545.000000,71047.000000,...,71046.000000,71046.000000,71046.000000,69803.000000,69803.000000,71047.000000,71047.000000,71047.000000,71047.000000,71047.000000
mean,3.201948e+06,58.852853,525.726250,46.862899,0.894817,40.087730,1.222315,-10.845304,-1.204730,6.009567,...,1.808617,1.561791,380.265631,31.375113,21.157715,0.037004,0.017918,0.050854,4.334229,0.053162
std,1.167965e+05,44.243566,530.136184,23.915000,2.197577,96.349599,9.081089,255.314034,38.770236,9.006656,...,1.336120,0.908280,254.294692,22.082195,23.917586,0.205823,0.141485,0.290444,3.137063,0.374988
min,3.000002e+06,-6.170000,0.000000,-11.000000,0.000000,0.000000,0.000000,-3875.000000,-1107.700000,0.000000,...,1.000000,1.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.100576e+06,33.640000,158.000000,30.000000,0.000000,0.000000,0.000000,-83.000000,-7.100000,0.700000,...,1.000000,1.000000,204.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,3.201478e+06,48.530000,366.000000,45.000000,0.250000,2.000000,0.000000,-5.000000,-0.300000,3.000000,...,1.000000,1.000000,330.000000,36.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
75%,3.304918e+06,71.030000,722.000000,60.000000,0.990000,41.000000,0.300000,66.000000,1.600000,7.700000,...,2.000000,2.000000,515.000000,48.000000,42.000000,0.000000,0.000000,0.000000,7.000000,0.000000
max,3.399994e+06,1223.380000,7668.000000,400.000000,159.390000,4321.000000,1112.400000,5192.000000,2483.500000,221.700000,...,28.000000,16.000000,1823.000000,99.000000,99.000000,4.000000,4.000000,35.000000,9.000000,25.000000


In [5]:
# Display basic information about the dataframe
print(f'Shape of the dataframe: {df.shape}')
print(f'Data types of each column:\n{df.dtypes}')

Shape of the dataframe: (71047, 58)
Data types of each column:
CustomerID                     int64
Churn                         object
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs  

#### Data Cleaning and Remove Null

In [6]:
# Count the number of missing values in each column of the dataframe
df.isnull().sum()

CustomerID                       0
Churn                        20000
MonthlyRevenue                 216
MonthlyMinutes                 216
TotalRecurringCharge           216
DirectorAssistedCalls          216
OverageMinutes                 216
RoamingCalls                   216
PercChangeMinutes              502
PercChangeRevenues             502
DroppedCalls                     0
BlockedCalls                     0
UnansweredCalls                  0
CustomerCareCalls                0
ThreewayCalls                    0
ReceivedCalls                    0
OutboundCalls                    0
InboundCalls                     0
PeakCallsInOut                   0
OffPeakCallsInOut                0
DroppedBlockedCalls              0
CallForwardingCalls              0
CallWaitingCalls                 0
MonthsInService                  0
UniqueSubs                       0
ActiveSubs                       0
ServiceArea                     28
Handsets                         1
HandsetModels       

In [7]:
#Drop CustomerID
df = df.drop('CustomerID', axis=1)

#drop all row containing 'Unknown' except
#df = df[df.iloc[:, ~df.columns.isin(['Homeownership'])].ne('Unknown').all(axis=1)]
df = df[df.iloc[:, ~df.columns.isin(['Homeownership', 'MaritalStatus'])].ne('Unknown').all(axis=1)]

# drop all rows containing NaNs
df = df.dropna()

#### Analyzed Cleaned dataset

In [8]:
# Display basic information about the dataframe
print(f'Shape of the dataframe: {df.shape}')
print(f'Data types of each column:\n{df.dtypes}')

Shape of the dataframe: (21489, 57)
Data types of each column:
Churn                         object
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea 

#### Preprocess the data

In [9]:
#get all categorical features
df_cat = df.select_dtypes(np.object)
cat_attributes = df_cat.columns
cat_attributes

Index(['Churn', 'ServiceArea', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'OwnsMotorcycle', 'HandsetPrice',
       'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode', 'Occupation',
       'MaritalStatus'],
      dtype='object')

In [10]:
#return unique value in categorical features
for feature in cat_attributes:
    print(feature ,': ',df[feature].unique())

Churn :  ['No' 'Yes']
ServiceArea :  ['BOSBOS978' 'ATHKIN423' 'CHIGRY219' 'OHICOL614' 'DETANN734' 'STLSTL314'
 'PITHOM412' 'NYCBRO917' 'LAXBEV310' 'OKCTUL918' 'DALDAL214' 'CHICHI773'
 'OHIDAY937' 'NYCMAN917' 'MINMIN612' 'NEVELC619' 'BIRBIR205' 'OKCLAW580'
 'LAXCDG310' 'APCFCH703' 'INDIND317' 'MINSTP612' 'DETPON248' 'NYCNEW201'
 'MIAWPB561' 'LAXVNY818' 'HARHAR860' 'FLNTAM813' 'KCYKCM816' 'SANSAN210'
 'APCSIL301' 'BOSBOS508' 'NEVLVS702' 'NCRCHA704' 'HARNOR203' 'ATLANE678'
 'OHICAN330' 'LAXRIV909' 'MILMIL414' 'NYCWHI914' 'AIRCHA843' 'NEVPOW619'
 'ATLNOR678' 'LAXPAS626' 'DETDET313' 'HWIHON808' 'PHIMER609' 'LAXMON323'
 'SFRSFR415' 'NYCNEW973' 'ATLKNO423' 'OHIPSV440' 'HOUHOU281' 'SFRSMO650'
 'BOSBOS781' 'BOSBOS617' 'DENGLD303' 'LAXCUL310' 'LAXPSG760' 'OMADES515'
 'SANAUS512' 'PHXSCO480' 'MIADEL561' 'SFRSCL408' 'FLNCLR813' 'PHIPHI215'
 'NYCNAS516' 'NYCQUE917' 'KCYKCK913' 'CHIRCK815' 'KCYTOP913' 'INHFTW219'
 'LAXONT909' 'MIAFTL954' 'FLNJAC904' 'OHICLE216' 'MIAMIA305' 'APCBAL410'
 'PHXPHX602' '


Summary of the analysis

1. CreditRating feature is ordinal
2. Occupation, PrizmCode, MaritalStatus need to convert to numerical using onehot encoding
3. ServiceArea will be removed as it identify the location. This info is biased and doesnt apply and support to new unknown data that potentially identify new area
3. The value of the Handset is mostly numerical value, we convert this value to integer type
4. The rest of the features is boolean with value 'Yes' and 'No', this will be need to convert to binary

In [11]:
## Convert CreditRating feature to ordinal

# Define the mapping of categories to integers
mapping = {'1-Highest': 1, '2-High': 2, '3-Good': 3, '4-Medium': 4, '5-Low': 5, '6-VeryLow': 6, '7-Lowest': 7}

# Map the categories to integers in the 'CreditRating' column
df['CreditRating'] = df['CreditRating'].replace(mapping)

In [12]:
## Convert Categorical using One Hot Encoding
cat_cols_encoding = ['PrizmCode', 'Occupation', 'MaritalStatus']
df = pd.get_dummies(df, columns=cat_cols_encoding, drop_first=True)

In [13]:
## Drop ServiceArea
#df = df.drop('ServiceArea', axis=1)

df = df.drop(['ServiceArea', 'TruckOwner', 'RVOwner'], axis=1)


In [14]:
## Convert HandsetPrice to integer
df['HandsetPrice'] = df['HandsetPrice'].astype(int)

In [15]:
## Convert Categorical to binary

# return list of features contain 'Yes' and 'No'
YesNoFeatures = cat_attributes.difference(['CreditRating','ServiceArea', 'PrizmCode', 'Occupation', 'MaritalStatus','Homeownership', 'TruckOwner', 'RVOwner'])
mapping = {'Yes': 1, 'No': 0}
df[YesNoFeatures] = df[YesNoFeatures].replace(mapping)

# convert Homeownership to binary
mapping = {'Known': 1, 'Unknown': 0}
df['Homeownership'] = df['Homeownership'].replace(mapping)

#### Split Feature and label data

In [16]:
X = df.drop('Churn', axis=1)
y = df[['Churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

### 3. Prediction Model Training and Evaluation

In this section, train various machine learning model and select the best performing model. 


The model use are as below:
1. Naive Bayes
2. Decision Tree
3. Logistic Regression
4. Support Vector Machine
5. Adaboost
6. Multi-Layer Perceptron Classifier
7. Voting Classifier

#### 3.1 Naive Bayes

In [17]:
# Define the objective function
def objective_function_NB(var_smoothing):
    model = GaussianNB(var_smoothing=var_smoothing)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return accuracy_score(y_test, y_pred)

# Define the search space
bounds_NB = {
    'var_smoothing': (1e-10, 1),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=objective_function_NB, pbounds=bounds_NB)

# Run the optimization
optimizer.maximize(init_points=10, n_iter=10)

# Get the best hyperparameters and the best score
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]

# Train best model
# Train the Naive Bayes classifier using the optimal hyperparameters
optimal_var_smoothing = best_params['var_smoothing']

model_NB = GaussianNB(var_smoothing=optimal_var_smoothing)
model_NB.fit(X_train, y_train)

# Evaluate Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model_NB.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   | var_sm... |
-------------------------------------
| 1         | 0.7284    | 0.3637    |
| 2         | 0.7284    | 0.8082    |
| 3         | 0.7284    | 0.3423    |
| 4         | 0.7284    | 0.51      |
| 5         | 0.7284    | 0.5558    |
| 6         | 0.7284    | 0.6491    |
| 7         | 0.7284    | 0.6587    |
| 8         | 0.7284    | 0.9959    |
| 9         | 0.7284    | 0.6253    |
| 10        | 0.7257    | 0.07237   |
| 11        | 0.7284    | 0.9104    |
| 12        | 0.7284    | 0.7451    |
| 13        | 0.7284    | 0.4431    |
| 14        | 0.7284    | 0.6525    |
| 15        | 0.7284    | 0.957     |
| 16        | 0.7284    | 0.8577    |
| 17        | 0.7284    | 0.7074    |
| 18        | 0.7284    | 0.4745    |
| 19        | 0.7284    | 0.9104    |
| 20        | 0.7284    | 0.5883    |
Accuracy: 0.7283620288506282
Precision: 0.5305112450714033
Recall: 0.7283620288506282
F1 Score: 0.6138890304413789


#### 3.2 Decision Tree

In [18]:
# Define the objective function
def DT_objective_function(max_depth, min_samples_split, min_samples_leaf):
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    min_samples_leaf = int(min_samples_leaf)

    classifier = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    return f1_score(y_test, y_pred)

# Define the search space
DT_bounds = {
    'max_depth': (3, 30),
    'min_samples_split': (2, 100),
    'min_samples_leaf': (1, 50),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=DT_objective_function, pbounds=DT_bounds)

# Run the optimization
optimizer.maximize(init_points=50, n_iter=50)

# Train best model
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]

optimal_max_depth = int(best_params['max_depth'])
optimal_min_samples_split = int(best_params['min_samples_split'])
optimal_min_samples_leaf = int(best_params['min_samples_leaf'])

DT_classifier = DecisionTreeClassifier(max_depth=optimal_max_depth, min_samples_split=optimal_min_samples_split, min_samples_leaf=optimal_min_samples_leaf)
DT_classifier.fit(X_train, y_train)

# Evaluate Model
'''
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(12, 8))
x1 = list(X.columns)
y1 = list(y.columns)

plot_tree(DT_classifier,filled=True)
plt.show()
'''

y_pred = DT_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | 0.2709    | 19.14     | 6.142     | 60.68     |
| 2         | 0.2079    | 10.16     | 14.22     | 68.91     |
| 3         | 0.2454    | 29.27     | 13.65     | 89.63     |
| 4         | 0.1865    | 7.09      | 15.02     | 75.47     |
| 5         | 0.2441    | 13.27     | 31.27     | 88.65     |
| 6         | 0.2679    | 23.74     | 2.216     | 67.14     |
| 7         | 0.2948    | 19.91     | 26.04     | 6.593     |
| 8         | 0.2418    | 27.99     | 23.11     | 98.62     |
| 9         | 0.2932    | 27.43     | 15.13     | 11.89     |
| 10        | 0.1978    | 9.994     | 43.64     | 23.84     |
| 11        | 0.2155    | 11.05     | 35.71     | 79.47     |
| 12        | 0.2249    | 19.61     | 38.24     | 97.26     |
| 13        | 0.208     | 10.48     | 8.608     | 68.65     |
| 14        | 0.2704    | 12.29     | 24.84     | 10.25     |
| 15    

#### 3.3 Logistic Regression

In [19]:
# Define the objective function
def objective_function_LR(C, penalty):
    C = C
    penalty = 'l1' if penalty < 0.5 else 'l2'
    
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear')
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return accuracy_score(y_test, y_pred)

# Define the search space
bounds_LR = {
    'C': (0.001, 10),
    'penalty': (0, 1),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=objective_function_LR, pbounds=bounds_LR)

# Run the optimization
optimizer.maximize(init_points=50, n_iter=20)

# Get the best hyperparameters and the best score
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]


# Train best model
optimal_C = best_params['C']
optimal_penalty = 'l1' if best_params['penalty'] < 0.5 else 'l2'

model_LR = LogisticRegression(C=optimal_C, penalty=optimal_penalty, solver='liblinear')

model_LR.fit(X_train, y_train)

# Evaluate Model
y_pred = model_LR.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   |     C     |  penalty  |
-------------------------------------------------
| 1         | 0.7278    | 6.924     | 0.5617    |
| 2         | 0.7278    | 2.332     | 0.7037    |
| 3         | 0.7275    | 2.115     | 0.4356    |
| 4         | 0.7277    | 9.003     | 0.6195    |
| 5         | 0.7278    | 4.264     | 0.8673    |
| 6         | 0.7273    | 8.513     | 0.8641    |
| 7         | 0.7275    | 0.8166    | 0.1418    |
| 8         | 0.7273    | 9.962     | 0.438     |
| 9         | 0.7277    | 1.418     | 0.3122    |
| 10        | 0.7274    | 1.345     | 0.9247    |
| 11        | 0.7278    | 0.2648    | 0.7426    |
| 12        | 0.7277    | 4.091     | 0.5044    |
| 13        | 0.7277    | 4.351     | 0.3353    |
| 14        | 0.7279    | 8.128     | 0.7485    |
| 15        | 0.7274    | 5.6       | 0.5544    |
| 16        | 0.7275    | 4.815     | 0.506     |
| 17        | 0.7278    | 6.582     | 0.7665    |
| 18        | 0.7275    | 2.895     | 0.2176    |


#### 3.4 SVC

In [20]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_scaler, y, test_size=0.2, random_state=0)

# Define the objective function
def objective_function_SVC(C, gamma):
    C = C
    gamma = gamma
    
    model = SVC(C=C, gamma=gamma, kernel='rbf', random_state=0)    
    
    model.fit(X_train2, y_train2)
    y_pred2 = model.predict(X_test2)

    return accuracy_score(y_test2, y_pred2)

# Define the search space
bounds_SVC = {
    'C': (0.1, 100),
    'gamma': (0.001, 1),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=objective_function_SVC, pbounds=bounds_SVC)

# Run the optimization
optimizer.maximize(init_points=10, n_iter=5)

# Get the best hyperparameters and the best score
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]


# Train best model
optimal_C = best_params['C']
optimal_gamma = best_params['gamma']

model_svc = SVC(C=optimal_C, gamma=optimal_gamma, kernel='rbf')

model_svc.fit(X_train2, y_train2)

# Evaluate Model
y_pred2 = model_svc.predict(X_test2)
accuracy = accuracy_score(y_test2, y_pred2)
precision = precision_score(y_test2, y_pred2, average='weighted')
recall = recall_score(y_test2, y_pred2, average='weighted')
f1 = f1_score(y_test2, y_pred2, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
| 1         | 0.7222    | 4.583     | 0.1271    |
| 2         | 0.7366    | 6.916     | 0.978     |
| 3         | 0.7336    | 77.26     | 0.4404    |
| 4         | 0.7199    | 47.08     | 0.1399    |
| 5         | 0.7343    | 18.87     | 0.483     |
| 6         | 0.7213    | 72.17     | 0.1472    |
| 7         | 0.7266    | 96.89     | 0.2369    |
| 8         | 0.7296    | 55.82     | 0.2652    |
| 9         | 0.7285    | 13.41     | 0.252     |
| 10        | 0.7345    | 38.91     | 0.5339    |
| 11        | 0.7366    | 6.916     | 0.9852    |
| 12        | 0.7366    | 8.46      | 0.9107    |
| 13        | 0.7366    | 36.29     | 0.9269    |
| 14        | 0.7378    | 33.9      | 0.001     |
| 15        | 0.7366    | 31.4      | 1.0       |
Accuracy: 0.737785016286645
Precision: 0.7019152303516439
Recall: 0.737785016286645
F1 Score: 0.6297321497076445


#### 3.5 Adaboost Classifier

In [21]:
# Define the objective function
def objective_function_adaboost(n_estimators, learning_rate):
    n_estimators = int(n_estimators)
    learning_rate = learning_rate
    
    model = AdaBoostClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return accuracy_score(y_test, y_pred)

# Define the search space
bounds_adaboost = {
    'n_estimators': (10, 200),
    'learning_rate': (0.001, 1),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=objective_function_adaboost, pbounds=bounds_adaboost)

# Run the optimization
optimizer.maximize(init_points=50, n_iter=50)

# Train best model
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]

# Train the AdaBoostClassifier model using the optimal hyperparameters
optimal_n_estimators = int(best_params['n_estimators'])
optimal_learning_rate = best_params['learning_rate']

model_adaboost = AdaBoostClassifier(n_estimators=optimal_n_estimators, learning_rate=optimal_learning_rate, random_state=0)

model_adaboost.fit(X_train, y_train)

# Evaluate Model
y_pred = model_adaboost.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
| 1         | 0.7394    | 0.2683    | 30.56     |
| 2         | 0.7438    | 0.8766    | 186.3     |
| 3         | 0.7417    | 0.4932    | 149.8     |
| 4         | 0.7394    | 0.6696    | 59.58     |
| 5         | 0.742     | 0.6949    | 197.5     |
| 6         | 0.741     | 0.5198    | 59.01     |
| 7         | 0.7438    | 0.9302    | 119.8     |
| 8         | 0.7434    | 0.7635    | 135.0     |
| 9         | 0.7417    | 0.7559    | 115.8     |
| 10        | 0.7408    | 0.3144    | 104.5     |
| 11        | 0.7399    | 0.315     | 90.2      |
| 12        | 0.7417    | 0.2727    | 129.6     |
| 13        | 0.742     | 0.7172    | 185.7     |
| 14        | 0.7422    | 0.8224    | 103.3     |
| 15        | 0.7406    | 0.2589    | 150.4     |
| 16        | 0.7408    | 0.297     | 195.4     |
| 17        | 0.7396    | 0.1125    | 96.86     |
| 18        | 0.7403    | 0.6779    | 28.54     |


#### 3.6 Multi-Layer Perceptron Classifier

In [22]:
# Define the objective function
def objective_function_MLP(hidden_layer_size, learning_rate, alpha):
    hidden_layer_size = int(hidden_layer_size)
    learning_rate = learning_rate
    alpha = alpha
    
    model = MLPClassifier(hidden_layer_sizes=(hidden_layer_size,),
                          learning_rate_init=learning_rate,
                          alpha=alpha)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    return accuracy_score(y_test, y_pred)

# Define the search space
bounds_MLP = {
    'hidden_layer_size': (10, 200),
    'learning_rate': (1e-5, 1e-1),
    'alpha': (1e-5, 1e-1),
}

# Set up the optimizer
optimizer = BayesianOptimization(f=objective_function_MLP, pbounds=bounds_MLP)

# Run the optimization
optimizer.maximize(init_points=60, n_iter=100)

# Train best model
best_params = optimizer.max["params"]
best_score = optimizer.max["target"]

# Train the MLPClassifier model using the optimal hyperparameters
optimal_hidden_layer_size = int(best_params['hidden_layer_size'])
optimal_learning_rate = best_params['learning_rate']
optimal_alpha = best_params['alpha']

model_MLP = MLPClassifier(hidden_layer_sizes=(optimal_hidden_layer_size,),
                      learning_rate_init=optimal_learning_rate,
                      alpha=optimal_alpha,
                      random_state=0)

model_MLP.fit(X_train, y_train)

# Evaluate Model
y_pred = model_MLP.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

|   iter    |  target   |   alpha   | hidden... | learni... |
-------------------------------------------------------------
| 1         | 0.7284    | 0.03468   | 82.36     | 0.07805   |
| 2         | 0.7268    | 0.06674   | 117.9     | 0.01129   |
| 3         | 0.7284    | 0.07083   | 132.2     | 0.05427   |
| 4         | 0.7284    | 0.001898  | 138.4     | 0.09819   |
| 5         | 0.7284    | 0.04879   | 41.85     | 0.07248   |
| 6         | 0.7249    | 0.05575   | 144.4     | 0.04831   |
| 7         | 0.7282    | 0.00783   | 97.25     | 0.06828   |
| 8         | 0.7282    | 0.06532   | 34.92     | 0.07321   |
| 9         | 0.7284    | 0.02146   | 150.0     | 0.09224   |
| 10        | 0.7284    | 0.05605   | 113.0     | 0.0824    |
| 11        | 0.7284    | 0.07368   | 185.6     | 0.08725   |
| 12        | 0.7163    | 0.07218   | 52.9      | 0.001649  |
| 13        | 0.7284    | 0.0606    | 20.82     | 0.06648   |
| 14        | 0.7224    | 0.02026   | 18.19     | 0.0007594 |
| 15    

#### 3.7 Voting Classifier

In [29]:
eclf1 = VotingClassifier(estimators=[('nb', model_NB), ('dt', DT_classifier),('lr', model_LR), ('adaboost', model_adaboost), ('mlp', model_MLP)], voting='soft')
eclf1.fit(X_train, y_train)
predictions = eclf1.predict(X_test)
print("Final Accuracy Score ")
print(accuracy_score(y_test, predictions))

Final Accuracy Score 
0.7269660307119591


#### Model Evaluation

In [30]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1


model1_metrics = evaluate_model(model_NB, X_test, y_test)
model2_metrics = evaluate_model(DT_classifier, X_test, y_test)
model3_metrics = evaluate_model(model_LR, X_test, y_test)
model4_metrics = evaluate_model(model_svc, X_test2, y_test2)
model5_metrics = evaluate_model(model_adaboost, X_test, y_test)
model6_metrics = evaluate_model(model_MLP, X_test, y_test)
model7_metrics = evaluate_model(eclf1, X_test, y_test)

metrics_df = pd.DataFrame(
    data=[model1_metrics, model2_metrics, model3_metrics, model4_metrics,model5_metrics,model6_metrics,model7_metrics],
    columns=['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    index=['Naive Bayes', 'Decision Tree', 'Logistic Regression', 'Support Vector Machine','Adaboost','Multi-Layer Perceptron Classifier','Voting Classifier']
)

print(metrics_df)

                                   Accuracy  Precision    Recall  F1 Score
Naive Bayes                        0.728362   0.530511  0.728362  0.613889
Decision Tree                      0.627734   0.626732  0.627734  0.627230
Logistic Regression                0.728478   0.669695  0.728478  0.624529
Support Vector Machine             0.737785   0.701915  0.737785  0.629732
Adaboost                           0.732434   0.692139  0.732434  0.643194
Multi-Layer Perceptron Classifier  0.727897   0.666471  0.727897  0.630739
Voting Classifier                  0.726966   0.669524  0.726966  0.647879


Upon a thorough analysis of the given performance metrics, it becomes evident that the Support Vector Machine (SVM) model outperforms its counterparts. Boasting the highest accuracy (0.737785016) and F1 Score (0.62973215), the SVM model demonstrates a remarkable balance between correct predictions and misclassifications.

Accuracy, a widely recognized metric for evaluating classifier performance, quantifies the proportion of correctly classified instances in relation to the total number of instances. Nevertheless, accuracy can be misleading when dealing with imbalanced class distributions. In such cases, the F1 Score emerges as a more reliable metric, as it accounts for both precision and recall. In this context, the SVM model's superior F1 Score (0.62973215) is particularly noteworthy.

Precision, which gauges the proportion of true positive instances out of instances predicted as positive, is crucial when false positives carry a high cost. The SVM model excels in this regard, with the highest precision (0.70191523) among the evaluated models. Moreover, the SVM model achieves the second-highest recall (0.737785016), only marginally lower than Logistic Regression (0.728478362). Recall is essential when minimizing false negatives is of utmost importance.

In conclusion, the Support Vector Machine model indisputably stands out among the given models, thanks to its superior accuracy, precision, and F1 Score, as well as its commendable recall. This exemplary performance establishes the SVM model as the most suitable choice for this particular problem, providing a solid foundation for accurate and reliable predictions.